# Pipelines de treinamento: NaiveBayes

Esse notebook contém as pipelines de treinamento usadas na obtenção do melhor modelo classificador do problema desenvolvido no EP2.

O modelo NaiveBayes aceita diversas parametrizações diferentes - veja documentação: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB - aqui exploramos duas: *alpha* e *force-alpha*. 

As pipelines treinadas são estratificadas em **features + MultinomialNB:force-alpha**.  
Isto quer dizer: existe uma pipeline para cada par de *feature* e *force-alpha* aqui explorados.   

As **features** que exploramos são:  

1. Bag of Words  
2. TF/TF-IDF  
3. CHAR NGrams  
4. Embeddings

Os **modelos de embeddings** que exploramos aqui são:  

1. BAAI bge-3
2. Google Gemma 300m 

PS: Os embeddings Gemma 300m eu ainda NÃO CONSEGUI GERAR. Então, não precisa testar a parte que roda com eles no EP.

Você pode encontrar uma execução já parametrizada do melhor modelo encontrado por essas pipelines no notebook model.ipynb, **que é nossa versão de entrega do EP2**.

## Bootstrap Imports

In [1]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path

### Teste de importação: Lib.utils do projeto

In [2]:
filedir = Path(os.getcwd())
base_path = filedir.resolve().parents[3]
sys.path.append(str(base_path))

from Lib.utils import printhello
printhello()

HELLO!


## Configura variáveis de execução

In [7]:
sep = ";"
dec = ","
quotech = "\""
encoding = "latin-1"


EP_dir = "EP2"
CSV_input_name = "ep2-train.csv"
path_to_archive = f"../../../../Traindata/{EP_dir}/{CSV_input_name}"


do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Path to csv input is:  ../../../../Traindata/EP2/ep2-train.csv


### Configure variáveis de reprodutibilidade

In [4]:
random_state = 12345

### Lista de melhores modelos

In [5]:
best_models_list = []

## Pré-tratamento de dados

### Importar dados do csv

In [8]:
df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

(43678, 2)
Index(['req_text', 'profession'], dtype='object')


### Embaralhamento dos dados

O .csv de entrada tem alto ordenamento dos inputs por classe. Carregá-los dessa maneira nos modelos p/ treinamento introduz viés, então é preciso embaralhar os dados para garantir randomicidade. 
As classes em sklearn.model_selection - como a StratfiedKFold usada mais a frente - implementam parâmetro shuffle="", que pode ser passado como True para embaralhar mais os dados.

Note que é importante também garantir a reprodutibilidade do embaralhamento, especificando um valor hardcoded (Neste caso random_state=100)

In [ ]:
print("Shape antes do shuffle:", df.shape)

df = df.sample(frac=1, random_state=random_state).reset_index(drop=True) #NAO MUDE random_state, essa variavel DEVE valer 12345, ou QUEBRARÁ REPRODUTIBILIDADE dos experimentos

print("Shape depois do shuffle:", df.shape)

Shape antes do shuffle: (43678, 2)
Shape depois do shuffle: (43678, 2)


### Limpeza dos dados

In [ ]:
from Lib.utils import clean_text
#def clean_text(text, do_lowercase: bool, rem_emails: bool, rem_urls: bool, normalize_whitespaces: bool):

df['req_text_cleaned'] = df['req_text'].apply(lambda row_text: clean_text(
        row_text, 
        do_lowercase=True, 
        rem_emails=True, 
        rem_urls=True, 
        normalize_whitespaces=True
    ))

df['req_text'] = df['req_text_cleaned']
df = df.drop(columns=['req_text_cleaned']) # Remove a coluna temporária

# Treino dos modelos NaiveBayes

## Imports

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.under_sampling import RandomUnderSampler

## Training Features: Bag of Words, TF-IDF, Word NGram

### Disable Warnings

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignora warnings de ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Ignora UserWarnings específicos de l1_ratio etc
warnings.filterwarnings("ignore", category=UserWarning)

### Feature: Bag of words 

#### Definição da Pipeline

In [ ]:
BoW_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
]) 

#### Treinamento c/ force_alpha=True

In [ ]:
BoW_parameters_T = {
    'kbest__k': [50, 100, 500, 1500, 4000, 10000, 20000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}

In [ ]:
BoW_classifier_T = GridSearchCV(BoW_pipeline, BoW_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
BoW_classifier_T.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", BoW_classifier_T.best_score_)
print("Melhores parâmetros:", BoW_classifier_T.best_params_)

best_models_list.append({
    "features": "BoW",
    "force-alpha": True,
    "accuracy": BoW_classifier_T.best_score_,
    "params": BoW_classifier_T.best_params_,
})

#### Treinamento c/ force_alpha=False

In [ ]:
BoW_parameters_F = {
    'kbest__k': [50, 100, 500, 1500, 4000, 10000, 20000, 'all'],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}

In [ ]:
BoW_classifier_F = GridSearchCV(BoW_pipeline, BoW_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
BoW_classifier_F.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", BoW_classifier_F.best_score_)
print("Melhores parâmetros:", BoW_classifier_F.best_params_)

best_models_list.append({
    "features": "BoW",
    "force-alpha": False,
    "accuracy": BoW_classifier_F.best_score_,
    "params": BoW_classifier_F.best_params_,
})

### Feature: TF-IDF

#### Definição da Pipeline

In [ ]:
TF_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
])

#### Treinamento c/ force_alpha=True

In [ ]:
TF_parameters_T = {
    'tfidf__use_idf': [True, False],
    'kbest__k': [50, 100, 500, 1500, 4000, 10000, 20000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}

In [ ]:
TF_classifier_T = GridSearchCV(TF_pipeline, TF_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
TF_classifier_T.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", TF_classifier_T.best_score_)
print("Melhores parâmetros:", TF_classifier_T.best_params_)

best_models_list.append({
    "features": "TF",
    "force-alpha": True,
    "accuracy": TF_classifier_T.best_score_,
    "params": TF_classifier_T.best_params_,
})

#### Treinamento c/ force_alpha=False

In [ ]:
TF_parameters_F = {
    'tfidf__use_idf': [True, False],
    'kbest__k': [50, 100, 500, 1500, 4000, 10000, 20000, 'all'],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}

In [ ]:
TF_classifier_F = GridSearchCV(TF_pipeline, TF_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
TF_classifier_F.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", TF_classifier_F.best_score_)
print("Melhores parâmetros:", TF_classifier_F.best_params_)

best_models_list.append({
    "features": "TF",
    "force-alpha": False,
    "accuracy": TF_classifier_F.best_score_,
    "params": TF_classifier_F.best_params_,
})

### Feature: CHAR Ngram

#### Definição da Pipeline

In [ ]:
NGram_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
]) 

#### Treinamento c/ force_alpha=True

In [ ]:
CHAR_NGram_parameters_T = { 
    'vect__ngram_range': [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12)],
    'vect__analyzer': ["char", "char_wb"],
    'kbest__k': [100, 200, 750, 4000, 10000, 25000],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}

In [ ]:
CHAR_NGram_classifier_T = GridSearchCV(NGram_pipeline, CHAR_NGram_parameters_T, 
                                       cv=10, n_jobs=2, scoring="accuracy", verbose=1, error_score = np.nan)
CHAR_NGram_classifier_T.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", CHAR_NGram_classifier_T.best_score_)
print("Melhores parâmetros:", CHAR_NGram_classifier_T.best_params_)

best_models_list.append({
    "features": "CHAR-NGram",
    "force-alpha": True,
    "accuracy": CHAR_NGram_classifier_T.best_score_,
    "params": CHAR_NGram_classifier_T.best_params_,
})

#### Treinamento c/ force_alpha=False

In [ ]:
CHAR_NGram_parameters_F = { 
    'vect__ngram_range': [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12)],
    'vect__analyzer': ["char", "char_wb"],
    'kbest__k': [100, 200, 750, 4000, 10000, 25000],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}

In [ ]:
CHAR_NGram_classifier_F = GridSearchCV(NGram_pipeline, CHAR_NGram_parameters_F, 
                                       cv=10, n_jobs=2, scoring="accuracy", verbose=1, error_score = np.nan)
CHAR_NGram_classifier_F.fit(df["req_text"].fillna(""), df["profession"].values)

In [ ]:
print("Melhor acurácia média:", CHAR_NGram_classifier_F.best_score_)
print("Melhores parâmetros:", CHAR_NGram_classifier_F.best_params_)

best_models_list.append({
    "features": "CHAR-NGram",
    "force-alpha": False,
    "accuracy": CHAR_NGram_classifier_F.best_score_,
    "params": CHAR_NGram_classifier_F.best_params_,
})

## Training Features: Embeddings

### Algoritmo & parâmetros

In [ ]:
# 2. Configurar o GridSearchCV diretamente no classificador KNN (sem o Pipeline wrapper)
nb = MultinomialNB()


nb_parameters_T = {
    'force_alpha': [True],
    'alpha': [1.0, 0.5, 0.1],
}

nb_parameters_F = {
    'force_alpha': [False],
    'alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}

### Feature: Embeddings BAAI-bge-3

#### Importação dos embeddings

In [ ]:
# 1. Gerar os dados X e Y
X_baai = np.load('../../Embeddings/npys/gen_baai_bge_3.ipynb')
y_baai = df["profession"].values

Gerando BAAI embeddings
Shape dos embeddings (X): (500, 1024)
embeddings gerados


#### Treinamento c/ force_alpha=True

In [ ]:
nb_baai_T = GridSearchCV(nb, nb_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
nb_baai_T.fit(X_baai, y_baai)

In [ ]:
print("Melhor acurácia média:", nb_baai_T.best_score_)
print("Melhores parâmetros:", nb_baai_T.best_params_)

best_models_list.append({
    "features": "embeddings baai-bge-3",
    "force-alpha": True,
    "accuracy": nb_baai_T.best_score_,
    "params": nb_baai_T.best_params_,
})

#### Treinamento c/ force_alpha=True

In [ ]:
nb_baai_F = GridSearchCV(nb, nb_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
nb_baai_F.fit(X_baai, y_baai)

In [ ]:
print("Melhor acurácia média:", nb_baai_F.best_score_)
print("Melhores parâmetros:", nb_baai_F.best_params_)

best_models_list.append({
    "features": "embeddings baai-bge-3",
    "force-alpha": False,
    "accuracy": nb_baai_F.best_score_,
    "params": nb_baai_F.best_params_,
})

### Feature: Embeddings Google Gemma-300m

#### Importação dos embeddings

In [ ]:
# 1. Gerar os dados X e Y
X_gemma = np.load('../../Embeddings/npys/gen_google_gemma_300m.ipynb')
y_gemma = df["profession"].values

#### Treinamento c/ force_alpha=True

In [ ]:
nb_gemma_T = GridSearchCV(nb, nb_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
nb_gemma_T.fit(X_gemma, y_gemma)

In [ ]:
print("Melhor acurácia média:", nb_gemma_T.best_score_)
print("Melhores parâmetros:", nb_gemma_T.best_params_)

best_models_list.append({
    "features": "embeddings google-gemma-300m",
    "force-alpha": True,
    "accuracy": nb_gemma_T.best_score_,
    "params": nb_gemma_T.best_params_,
})

#### Treinamento c/ force_alpha=False

In [ ]:
nb_gemma_F = GridSearchCV(nb, nb_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
nb_gemma_F.fit(X_gemma, y_gemma)

In [ ]:
print("Melhor acurácia média:", nb_gemma_F.best_score_)
print("Melhores parâmetros:", nb_gemma_F.best_params_)

best_models_list.append({
    "features": "embeddings google-gemma-300m",
    "force-alpha": False,
    "accuracy": nb_gemma_F.best_score_,
    "params": nb_gemma_F.best_params_,
})

# Seleciona melhores parâmetros

In [ ]:
best_score = -1
best = 0
for idx, candidate in enumerate(best_models_list):
    if candidate["accuracy"] > best_score:
        best = idx
        best_score = candidate["accuracy"]

print(f"O melhor classificador encontrado pelas pipelines é -->    feature={best_models_list[best]["features"]} + force_alpha={best_models_list[best]["force_alpha"]}\n")
print(f"Melhor acucácia encontrada:  {best_models_list[best]['accuracy']}")
print(f"Melhores parametros encontrados:  {best_models_list[best]['params']}")